## CoFrNet Example Notebook

###### Imports and Seeds

In [ ]:
# Imports and Seeds
from Customized_Linear_Classes import CustomizedLinearFunction
from Customized_Linear_Classes import CustomizedLinear
from utils import generate_connections
from utils import process_data
from utils import train
from utils import OnlyTabularDataset
from CoFrNet import CoFrNet_Model
from CoFrNet import generate_connections
from CoFrNet import CoFrNet_Explainer
from torch.utils.data import DataLoader
from tqdm import tqdm
import numpy as np 
from torch.utils.data import Dataset
import torch # import main library
import torch.nn as nn # import modules
from torch.autograd import Function # import Function to create custom activations
from torch.nn.parameter import Parameter # import Parameter to create custom activations with learnable parameters
import torch.nn.functional as F # import torch functions
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split  
import torch.optim as optim
import random
import kaggle



seed_num = 5
random.seed(seed_num)
torch.manual_seed(seed_num)
np.random.seed(seed_num)

###### Setting Up CoFrNet

In [ ]:
'''
input_size: number of features in your dataset
output_size: number of classification categories (number of outputs)
cofrnet_version: CoFrNet Variant to be created (see paper for details)
network depth: depth of selected CoFrNet Variant. For example, if you select diag_ladder_of_ladder_combined, 
                the diagonalized section of the network will have depth 13 and the network will explore up to 
                order 13 interactions. 
'''
network_depth = 13
input_size = 10 #10 for telescopeMagic #40 for waveform #23 for creditcard
output_size = 2 #2 for telescope #3 for waveform #2 for creditcard
cofrnet_version = "diag_ladder_of_ladder_combined"
#Create CoFrNet
model = CoFrNet_Model(generate_connections(network_depth, 
                                            input_size, 
                                            output_size, 
                                            cofrnet_version))


###### Get Data

In [ ]:
'''
Download dataset from Kaggle
'''
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()

api.dataset_download_files('abhinand05/magic-gamma-telescope-dataset', path=".", unzip = True)



data_filename= 'telescope_data.csv'
first_column_csv = 1
last_column_csv = -1


'''
Use process_data method in utils.py to get tensors for training, validation, testing
'''
#Get Data
tensor_x_train, tensor_y_train, tensor_x_val, tensor_y_val, tensor_x_test, y_test = process_data(data_filename= data_filename, 
                                                                                                first_column_csv = first_column_csv, 
                                                                                                last_column_csv = last_column_csv)

'''
Create a Dataloader
'''
train_dataset = OnlyTabularDataset(tensor_x_train, 
                                    tensor_y_train)

batch_size = 100
dataloader = DataLoader(train_dataset, batch_size)


###### Training

In [ ]:
'''
Use Train Module from Utils to Train
'''
train(model, dataloader, output_size)

###### Print Accuracies, Continued Fraction, Importances

In [ ]:
'''
Print Accuracy of Network
'''
explainer = CoFrNet_Explainer(model)
explainer.print_accuracy(tensor_x_test, y_test)